In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import keras
from keras import metrics
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation
import os
import pickle
import cv2
import imutils
from imutils.contours import sort_contours

from src import fonctions

Using TensorFlow backend.


In [2]:
# Chemins du projet
DATA_ROOT = 'data/'
MODELE_PATH = DATA_ROOT + 'model.h5'
TRAINING_PATH = DATA_ROOT + 'train'
TESTING_PATH = DATA_ROOT + 'test'

# Autres paramètres
batch_size = 32
num_classes = 100
epochs = 35
img_size = 28
input_shape = (img_size, img_size, 1)

In [3]:
# fonctions.import_fichier_train()

In [4]:
# fonctions.import_fichier_test()

In [5]:


### ImageDataGenerator génère des lots de données d'image vectorielles, convertissant les coefficients RVB compris entre 0 et 255 en valeurs cibles comprises entre 0 et 1 par mise à l'échelle avec un facteur de 1/255 à l' aide de la remise à l' échelle ###

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    ### shear_range est utilisé pour appliquer de manière aléatoire des transformations de cisaillement ###
    shear_range = 0.2,
    ### zoom_range est utilisé pour zoomer aléatoirement à l'intérieur des images ###           
    zoom_range = 0.2,
    ### horizontal_flip est utilisé pour retourner au hasard la moitié des images horizontalement ###            
    horizontal_flip = True
)      

test_datagen = ImageDataGenerator(rescale = 1./255)


### J'importe les images une par une à partir des répertoires en utilisant .flow_from_directory et y appliquons ImageDataGenerator ###

train_generator = train_datagen.flow_from_directory(
    ### Choix de mon repertoire ###
    directory = TRAINING_PATH,
    ### Je converti les images de leur taille d'origine à notre target_size ###                    
    target_size = (img_size,img_size),
    ### Nombre batch_size qui fait référence au nombre d'exemples d'entraînement utilisés dans une itération ###                                      
    batch_size = batch_size,
    ### Je definis le class_mode sur "catégorical" indiquant que nous avons plusieurs classes (a à z) à prédire ###          
    class_mode = "categorical",
    ### Je choisis le color_mode "grayscale", indiquant que nous trvaillons sur une image en noir et blanc
    color_mode = "grayscale"                                

)

test_generator = test_datagen.flow_from_directory(
    directory = TESTING_PATH,
    target_size = (img_size,img_size),
    batch_size = batch_size,
    class_mode = "categorical",
    color_mode = "grayscale"

)



Found 11600 images belonging to 100 classes.
Found 48400 images belonging to 100 classes.


In [6]:
### Création d'un modèle séquentiel qui permet de définir l'architecture CNN couche par couche à l'aide de la fonction .add .Nous ajoutons d'abord une couche de convolution avec 32 filtres de taille 3X3 sur les images d'entrée et la passons à travers la fonction d'activation 'relu'.Nous effectuons ensuite des opérations MaxPooling en utilisant un pool de taille 2X2 ###
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = input_shape, activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

### Ces couches sont ensuite répétées à nouveau pour améliorer les performances du modèle ###

model.add(Conv2D(32, (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

### Enfin, nous aplatissons notre matrice résultante et la passons à travers une couche dense composée de 128 nœuds. Celui-ci est ensuite connecté à la couche de sortie constituée de 26 nœuds, chaque nœud représentant un alphabet ###

model.add(Flatten())
model.add(Dense(units = 128, activation = "relu"))
model.add(Dense(units = 100, activation = "softmax"))            ### Activation softmax qui convertit les scores en une distribution de probabilité normalisée, et                                                                   le nœud avec la probabilité la plus élevée est sélectionné comme sortie ###

### Une fois notre architecture CNN définie, nous compilons le modèle à l'aide de l'optimiseur Adam ###
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               102528    
_________________________________________________________________
dense_2 (Dense)              (None, 100)             

In [7]:
### Je décide de créer 25 répétitions, et j'ai X2 les steps_per_epoch pour augmenter notre précision ###

entrainement = model.fit_generator(
    train_generator,
    steps_per_epoch = batch_size,
    epochs = epochs,
    validation_data = test_generator,
    validation_steps = batch_size
)

score = model.evaluate(train_generator, verbose=0)
print("Test de perte:", score[0])
print("Test de précision:", score[1])
print("Enregistrement du modèle...")
model.save(MODELE_PATH)
print("Modèle enregistré!")


Epoch 1/35
32/32 [==============================] - 22s 675ms/step - loss: 4.5336 - accuracy: 0.0498 - val_loss: 4.6909 - val_accuracy: 9.7656e-04
Epoch 2/35
32/32 [==============================] - 16s 505ms/step - loss: 4.4959 - accuracy: 0.0537 - val_loss: 4.6993 - val_accuracy: 9.7656e-04
Epoch 3/35
32/32 [==============================] - 16s 514ms/step - loss: 4.5009 - accuracy: 0.0469 - val_loss: 4.6226 - val_accuracy: 0.0039
Epoch 4/35
32/32 [==============================] - 15s 483ms/step - loss: 4.5197 - accuracy: 0.0596 - val_loss: 4.7612 - val_accuracy: 0.0029
Epoch 5/35
32/32 [==============================] - 16s 489ms/step - loss: 4.4662 - accuracy: 0.0645 - val_loss: 4.7022 - val_accuracy: 0.0029
Epoch 6/35
32/32 [==============================] - 17s 527ms/step - loss: 4.4752 - accuracy: 0.0791 - val_loss: 4.6613 - val_accuracy: 0.0020
Epoch 7/35
32/32 [==============================] - 16s 504ms/step - loss: 4.4412 - accuracy: 0.0723 - val_loss: 4.8222 - val_accurac